Count all files in the consvmets data
- load their csv
- assign each folder to a dataset
- count audio files

In [1]:
import pandas as pd
import csv
import os

In [12]:
# define master folder
master_folder = r'D:\CV4Eco\Data\Conservation Metrics data\cmi_soundfiles_20230719\src'

# folder to write results to
results_folder = r'C:\Users\ben07\OneDrive - University College London\Google\work\data_wrangling\csvs'

In [3]:
# list all dirs in master folder
def list_directories(master_folder):
    with os.scandir(master_folder) as entries:
        for entry in entries:
            if entry.is_dir():
                print(entry.name)

# Run
dataset_list = list_directories(master_folder)

Atlantic
CMI_PalmyraReef_2023_R1_FLAC
CMI_Palmyra_HydroMoth_2021_R1
CRIMP_Kaneohe_2022_R1
CRIMP_Kaneohe_2023_R1_FLAC
HIMB_Halos_2022_R1
HIMB_Halos_2022_R2
HIMB_Halos_2022_R3
HIMB_Halos_2022_R4
HIMB_Halos_2022_R5
HIMB_Halos_2023_R1
HIMB_Halos_2023_R2
HIMB_Halos_2023_R3_FLAC
HIMB_Halos_2023_R4_FLAC
mozambique
NOAA_SanctSound_FLKeys
Rice_Pacific
SOS_STX_2022_R6_FLAC
SOS_STX_2022_R7_FLAC
SOS_STX_ButlerBay
SOS_Thailand_2023_R1_FLAC
SOS_ZAN_ChumbeIsland


Now we bin folders into specific datasets

# to do here!
- add all desired metadata keys to datasets dict. have None for empties
- this will create a csv with all the info i want. maybe create a mini for just each dataset. send this to jill and ask if they can fill any metadata gaps.
- get this to github

In [ ]:
# Define the datasets
datasets = {
    "palmyra_hydromoth": {"folders": ["CMI_Palmyra_HydroMoth_2021_R1"], "data_owner": "CMI", "hydrophone_type": "TypeA"},
    "palmyra_other": ["CMI_PalmyraReef_2023_R1_FLAC"],
    "hawaii_kaneohe": ["CRIMP_Kaneohe_2022_R1", "CRIMP_Kaneohe_2023_R1_FLAC"],
    "hawaii_himb": ["HIMB_Halos_2022_R1",
                    "HIMB_Halos_2022_R2",
                    "HIMB_Halos_2022_R3",
                    "HIMB_Halos_2022_R4",
                    "HIMB_Halos_2022_R5",
                    "HIMB_Halos_2023_R1",
                    "HIMB_Halos_2023_R2",
                    "HIMB_Halos_2023_R3_FLAC",
                    "HIMB_Halos_2023_R4_FLAC"],
    "mozambique": ["mozambique"],
    "florida_noaa": ["NOAA_SanctSound_FLKeys"],
    "philippines_rice": ["Rice_Pacific"],
    "usvi": ["SOS_STX_2022_R6_FLAC","SOS_STX_2022_R7_FLAC","SOS_STX_ButlerBay"],
    "thailand": ["SOS_Thailand_2023_R1_FLAC"],
    "zanzibar": ["SOS_ZAN_ChumbeIsland"],
    "belize": ["Atlantic/Belize"],
    "bermuda": ["Atlantic/Bermuda"],
    "mexico": ["Atlantic/Cozumel"]
}

In [10]:
# Define the datasets
datasets = {
    "palmyra_hydromoth": ["CMI_Palmyra_HydroMoth_2021_R1"],
    "palmyra_other": ["CMI_PalmyraReef_2023_R1_FLAC"],
    "hawaii_kaneohe": ["CRIMP_Kaneohe_2022_R1", "CRIMP_Kaneohe_2023_R1_FLAC"],
    "hawaii_himb": ["HIMB_Halos_2022_R1",
                    "HIMB_Halos_2022_R2",
                    "HIMB_Halos_2022_R3",
                    "HIMB_Halos_2022_R4",
                    "HIMB_Halos_2022_R5",
                    "HIMB_Halos_2023_R1",
                    "HIMB_Halos_2023_R2",
                    "HIMB_Halos_2023_R3_FLAC",
                    "HIMB_Halos_2023_R4_FLAC"],
    "mozambique": ["mozambique"],
    "florida_noaa": ["NOAA_SanctSound_FLKeys"],
    "philippines_rice": ["Rice_Pacific"],
    "usvi": ["SOS_STX_2022_R6_FLAC","SOS_STX_2022_R7_FLAC","SOS_STX_ButlerBay"],
    "thailand": ["SOS_Thailand_2023_R1_FLAC"],
    "zanzibar": ["SOS_ZAN_ChumbeIsland"],
    "belize": ["Atlantic/Belize"],
    "bermuda": ["Atlantic/Bermuda"],
    "mexico": ["Atlantic/Cozumel"]
}

In [25]:
# datasets = { "bermuda": {"folders":["Atlantic/Bermuda"], "data_owner": "CMI", "hydrophone_type": "TypeA"},
#     "mexico": {"folders":["Atlantic/Cozumel"], "data_owner": "who", "hydrophone_type": "TypeA"}
# }

now start storing filepaths for audio files in each dataset

In [33]:
def collect_audio_files(folder_path, dataset_name, meta_data, file_list):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(('.flac', '.wav')):
                rel_path = os.path.relpath(root, start="D:\\CV4Eco\\Data\\Conservation Metrics data\\cmi_soundfiles_20230719")
                full_rel_path = os.path.join(rel_path, file)
                file_info = {"filepath": full_rel_path, "dataset": dataset_name}
                file_info.update(meta_data)  # add extra info like 'data_owner', 'hydrophone_type', etc.
                file_list.append(file_info)


file_list = []

for dataset, attributes in datasets.items():
    folders = attributes.get("folders", [])
    meta_data = {k: v for k, v in attributes.items() if k != "folders"}  # Exclude 'folders' from meta_data

    for folder in folders:
        folder_path = os.path.join(master_folder, folder)
        collect_audio_files(folder_path, dataset, meta_data, file_list)

# Create DataFrame
df = pd.DataFrame(file_list)

# Print dataset counts
dataset_counts = df.groupby('dataset').size()
print("Number of audio files per dataset:")
print(dataset_counts)

# Print total count
print(f"Total audio files: {len(df)}")

# Save DataFrame to CSV
df.to_csv(results_folder + "/consvmets_file_list.csv", index=False)

Number of audio files per dataset:
dataset
bermuda    3766
mexico     1086
dtype: int64
Total audio files: 4852


In [31]:
df

,filepath,dataset,data_owner,hydrophone_type
0,src\Atlantic\Bermuda\805347352-Sound Files\805...,bermuda,CMI,TypeA
1,src\Atlantic\Bermuda\805347352-Sound Files\805...,bermuda,CMI,TypeA
2,src\Atlantic\Bermuda\805347352-Sound Files\805...,bermuda,CMI,TypeA
3,src\Atlantic\Bermuda\805347352-Sound Files\805...,bermuda,CMI,TypeA
4,src\Atlantic\Bermuda\805347352-Sound Files\805...,bermuda,CMI,TypeA
...,...,...,...,...
4847,src\Atlantic\Cozumel\Cozumel_wavs\AMAR394_1_03...,mexico,who,TypeA
4848,src\Atlantic\Cozumel\Cozumel_wavs\AMAR394_1_03...,mexico,who,TypeA
4849,src\Atlantic\Cozumel\Cozumel_wavs\AMAR394_1_03...,mexico,who,TypeA
4850,src\Atlantic\Cozumel\Cozumel_wavs\AMAR394_1_03...,mexico,who,TypeA
